In [1]:
import pandas as pd
import utils
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import joblib

In [2]:
input_csv = 'treino.csv'

In [3]:
df = pd.read_csv(input_csv, sep=';')

In [4]:
df.head()

,id,mensagem,tipo
0,1,bom dia,saudacao
1,2,boa tarde,saudacao
2,3,boa noite,saudacao
3,4,olá,saudacao
4,5,oi,saudacao


In [5]:
df.groupby('tipo').count()

,id,mensagem
tipo,,
avaliacoes,19,19
pagamento,18,18
saudacao,19,19
tutoria,19,19


In [6]:
df['mensagem_tratada'] = df.mensagem.apply(utils.to_lower_remove_accents)

In [7]:
df

,id,mensagem,tipo,mensagem_tratada
0,1,bom dia,saudacao,bom dia
1,2,boa tarde,saudacao,boa tarde
2,3,boa noite,saudacao,boa noite
3,4,olá,saudacao,ola
4,5,oi,saudacao,oi
...,...,...,...,...
70,71,não entendi a abordagem do professor,tutoria,nao entendi a abordagem do professor
71,72,não entendi o conteúdo,tutoria,nao entendi o conteudo
72,73,a explicação ficou confusa,tutoria,a explicacao ficou confusa
73,74,não consigo resolver os exercicios,tutoria,nao consigo resolver os exercicios


## Classificador

In [8]:
vectorizer = TfidfVectorizer()

In [9]:
corpus = df['mensagem_tratada'].values

In [10]:
vectorizer.fit(corpus)
# joblib.dump(vectorizer, 'vectorizer.joblib')

X = vectorizer.transform(corpus)

In [11]:
X

<75x104 sparse matrix of type '<class 'numpy.float64'>'
	with 338 stored elements in Compressed Sparse Row format>

In [12]:
y = df['tipo'].values

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

## Usando o modelo de classificação

In [14]:
clf = LogisticRegression(random_state=0, class_weight='balanced', multi_class='ovr', solver='lbfgs').fit(X_train, y_train)
clf

LogisticRegression(class_weight='balanced', multi_class='ovr', random_state=0)

In [15]:
joblib.dump(clf, 'classifier.joblib')

['classifier.joblib']

In [16]:
y_pred = clf.predict(X_test)

In [17]:
y_pred

array(['tutoria', 'tutoria', 'saudacao', 'saudacao', 'avaliacoes',
       'tutoria', 'avaliacoes', 'tutoria', 'tutoria', 'tutoria',
       'avaliacoes', 'pagamento', 'pagamento', 'saudacao', 'saudacao',
       'avaliacoes', 'saudacao', 'avaliacoes', 'pagamento', 'tutoria',
       'pagamento', 'pagamento', 'saudacao', 'pagamento', 'pagamento'],
      dtype=object)

In [18]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

  avaliacoes       1.00      1.00      1.00         5
   pagamento       0.86      1.00      0.92         6
    saudacao       1.00      0.75      0.86         8
     tutoria       0.71      0.83      0.77         6

    accuracy                           0.88        25
   macro avg       0.89      0.90      0.89        25
weighted avg       0.90      0.88      0.88        25



In [19]:
print(confusion_matrix(y_test, y_pred))

[[5 0 0 0]
 [0 6 0 0]
 [0 0 6 2]
 [0 1 0 5]]


In [20]:
import chatbot

In [21]:
%run chatbot.py

Oi, eu sou o Chatbot. Em que posso ajudar?

... boleto

Para obter mais detalhes sobre pagamentos e segundas vias, acesse:
www.minhaempresa.com.br/pagamentos
... duvidas sobre a materia

Estou encaminhando sua dúvida para a tutoria.
... sair
